# Google Cloud autentikointi:


In [ ]:

!pip install tqdm
!pip install rembg



from tqdm import tqdm
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas as pd
from skimage import color
from rembg import remove
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
import webcolors
from colorsys import rgb_to_hsv


from google.colab import auth
auth.authenticate_user()
print('Authenticated')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.2 MB/s eta 0:00:00
Authenticated


## Haetaan muutettava data df muuttujaksi


In [ ]:
%%bigquery df --project touhua-001
SELECT *
FROM `touhua-001.qa_dm_kesko.d_product`
WHERE
ecom_category LIKE "Juomat"
-- AND
-- ecom_subCategory <> 'Oluet'
AND
imageUrl IS NOT NULL
ORDER BY RAND()
LIMIT 300;




Could not save output to variable 'df'.

ERROR:
 ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7a6ebac90e80>)


In [ ]:
df = pd.DataFrame(df)

df.head()

,timestamp,ean,productName,contentSize,contentUnit,brand,segmentId,ecom_categoryPath,ecom_category,ecom_subCategory,...,segmentName,isAlcohol,percentage,sugar,carbohydrates,protein,lactose,salt,energyKcal,imageUrl
0,2024-05-15 02:47:05.621000+00:00,7310090165538,Bob Lättdryck citron&lime 2dl,0.20,l,None,3301,juomat/mehut/kerta-annosmehut,Juomat,Mehut ja smoothiet,...,Pieni juoma ja nektari lämmin,False,None,6.5,6.5,NaN,NaN,NaN,30.0,https://public.keskofiles.com/f/k-ruoka/produc...
1,2024-06-10 05:22:26.451000+00:00,8008440412188,"Peroni Nastro Azzurro 4,6% 0,33l",0.33,l,PERONI,5232,juomat/oluet/ulkomaiset-oluet,Juomat,Oluet,...,"Ulkomainen lager-olut enint. 5,5 % tlk",True,4.600000000,0.0,0.0,0.0,NaN,0.0,0.0,https://public.keskofiles.com/f/k-ruoka/produc...
2,2024-01-09 08:47:00.783000+00:00,6429830077626,Herra Papusen Konkarin Kahvi 200g suodatinkahvi,0.20,kg,HERRA PAPUSEN,3409,juomat/kahvi-ja-suodatinpussit/muut-kahvit,Juomat,Kahvi ja suodatinpussit,...,Muut kahvit,False,None,NaN,NaN,NaN,NaN,NaN,NaN,https://public.keskofiles.com/f/k-ruoka/produc...
3,2024-06-10 04:57:51.372000+00:00,5021991944567,"Clipper Hiilihapoton juoma Sitruuna, inkivääri...",0.25,l,CLIPPER,3349,juomat/mehut/mehut-lampimasta,Juomat,Mehut ja smoothiet,...,Muut juomat ja nektarit lämmin,False,None,4.9,5.0,0.2,NaN,0.0,23.0,https://public.keskofiles.com/f/k-ruoka/produc...
4,2024-06-10 05:17:49.822000+00:00,6430052491114,"Iso-Kallan Savu Kekri olut 5,5% 0,33l",0.33,l,ISO-KALLAN PANIMO,5220,juomat/oluet/kotimaiset-pienpanimo--ja-erikois...,Juomat,Oluet,...,Kotimainen pienpanimo- ja erikoisolut enint. 5...,True,5.500000000,NaN,NaN,NaN,NaN,NaN,NaN,https://public.keskofiles.com/f/k-ruoka/produc...


## Värien määrittely ja niiden luokittelu/normalisointi

In [ ]:
colors_dict = {
"0048BA":"Absolute Zero","B0BF1A":"Acid green","7CB9E8":"Aero","C9FFE5":"Aer o blue","B284BE":"African violet","72A0C1":"Air superiority blue","EDEAE0":"Alabaster","F0F8FF":"Alice blue","C46210":"Alloy orange","EFDECD":"Almond","E52B50":"Amaranth","9F2B68":"Amaranth (M&P)","F19CBB":"Amaranth pink","AB274F":"Amaranth purple","D3212D":"Amaranth red","3B7A57":"Amazon","FFBF00":"Amber","FF7E00":"Amber (SAE/ECE)","9966CC":"Amethyst","A4C639":"Android green","CD9575":"Antique brass","665D1E":"Antique bronze","915C83":"Antique fuchsia","841B2D":"Antique ruby","FAEBD7":"Antique white","008000":"Ao (English)",
"8DB600":"Apple green","FBCEB1":"Apricot","00FFFF":"Aqua","7FFFD4":"Aquamarine","D0FF14":"Arctic lime","4B5320":"Army green","8F9779":"Artichoke","E9D66B":"Arylide yellow","B2BEB5":"Ash gray","87A96B":"Asparagus","FF9966":"Atomic tangerine","A52A2A":"Auburn","FDEE00":"Aureolin","568203":"Avocado","007FFF":"Azure","F0FFFF":"Azure (X11/web color)","89CFF0":"Baby blue","A1CAF1":"Baby blue eyes","F4C2C2":"Baby pink","FEFEFA":"Baby powder","FF91AF":"Baker-Miller pink","FAE7B5":"Banana Mania","DA1884":"Barbie Pink","7C0A02":"Barn red","848482":"Battleship grey","BCD4E6":"Beau blue",
"9F8170":"Beaver","F5F5DC":"Beige","2E5894":"B'dazzled blue","9C2542":"Big dip o’ruby","FFE4C4":"Bisque","3D2B1F":"Bistre","967117":"Bistre brown","CAE00D":"Bitter lemon","BFFF00":"Bitter lime","FE6F5E":"Bittersweet","BF4F51":"Bittersweet shimmer","000000":"Black","3D0C02":"Black bean","1B1811":"Black chocolate","3B2F2F":"Black coffee","54626F":"Black coral","3B3C36":"Black olive","BFAFB2":"Black Shadows","FFEBCD":"Blanched almond","A57164":"Blast-off bronze","318CE7":"Bleu de France","ACE5EE":"Blizzard blue","FAF0BE":"Blond","660000":"Blood red","0000FF":"Blue","1F75FE":"Blue (Crayola)",
"0093AF":"Blue (Munsell)","0087BD":"Blue (NCS)","0018A8":"Blue (Pantone)","333399":"Blue (pigment)","0247FE":"Blue (RYB)","A2A2D0":"Blue bell","6699CC":"Blue-gray","0D98BA":"Blue-green","064E40":"Blue-green (color wheel)","5DADEC":"Blue jeans","126180":"Blue sapphire","8A2BE2":"Blue-violet","7366BD":"Blue-violet (Crayola)","4D1A7F":"Blue-violet (color wheel)","5072A7":"Blue yonder","3C69E7":"Bluetiful","DE5D83":"Blush","79443B":"Bole","E3DAC9":"Bone","006A4E":"Bottle green","87413F":"Brandy","CB4154":"Brick red","66FF00":"Bright green","D891EF":"Bright lilac","C32148":"Bright maroon","1974D2":"Bright navy blue",
"FFAA1D":"Bright yellow (Crayola)","FF55A3":"Brilliant rose","FB607F":"Brink pink","004225":"British racing green","CD7F32":"Bronze","88540B":"Brown","AF6E4D":"Brown sugar","1B4D3E":"Brunswick green","7BB661":"Bud green","F0DC82":"Buff","800020":"Burgundy","DEB887":"Burlywood","A17A74":"Burnished brown","CC5500":"Burnt orange","E97451":"Burnt sienna","8A3324":"Burnt umber","BD33A4":"Byzantine","702963":"Byzantium","536872":"Cadet","5F9EA0":"Cadet blue","A9B2C3":"Cadet blue (Crayola)","91A3B0":"Cadet grey","006B3C":"Cadmium green","ED872D":"Cadmium orange","E30022":"Cadmium red","FFF600":"Cadmium yellow",
"A67B5B":"Café au lait","4B3621":"Café noir","A3C1AD":"Cambridge blue","C19A6B":"Camel","EFBBCC":"Cameo pink","FFFF99":"Canary","FFEF00":"Canary yellow","FF0800":"Candy apple red","E4717A":"Candy pink","00BFFF":"Capri","592720":"Caput mortuum","C41E3A":"Cardinal","00CC99":"Caribbean green","960018":"Carmine","D70040":"Carmine (M&P)","FFA6C9":"Carnation pink","B31B1B":"Carnelian","56A0D3":"Carolina blue","ED9121":"Carrot orange","00563F":"Castleton green","703642":"Catawba","C95A49":"Cedar Chest","ACE1AF":"Celadon","007BA7":"Celadon blue","2F847C":"Celadon green","B2FFFF":"Celeste",
"246BCE":"Celtic blue","DE3163":"Cerise","007BA7":"Cerulean","2A52BE":"Cerulean blue","6D9BC3":"Cerulean frost","1DACD6":"Cerulean (Crayola)","007AA5":"CG blue","E03C31":"CG red","F7E7CE":"Champagne","F1DDCF":"Champagne pink","36454F":"Charcoal","232B2B":"Charleston green","E68FAC":"Charm pink","DFFF00":"Chartreuse (traditional)","7FFF00":"Chartreuse (web)","FFB7C5":"Cherry blossom pink","954535":"Chestnut","DE6FA1":"China pink","A8516E":"China rose","AA381E":"Chinese red","856088":"Chinese violet","FFB200":"Chinese yellow","7B3F00":"Chocolate (traditional)","D2691E":"Chocolate (web)","FFA700":"Chrome yellow","98817B":"Cinereous",
"E34234":"Cinnabar","CD607E":"Cinnamon Satin","E4D00A":"Citrine","9FA91F":"Citron","7F1734":"Claret","0047AB":"Cobalt blue","D2691E":"Cocoa brown","6F4E37":"Coffee","B9D9EB":"Columbia Blue","F88379":"Congo pink","8C92AC":"Cool grey","B87333":"Copper","DA8A67":"Copper (Crayola)","AD6F69":"Copper penny","CB6D51":"Copper red","996666":"Copper rose","FF3800":"Coquelicot","FF7F50":"Coral","F88379":"Coral pink","893F45":"Cordovan","FBEC5D":"Corn","6495ED":"Cornflower blue","FFF8DC":"Cornsilk","2E2D88":"Cosmic cobalt","FFF8E7":"Cosmic latte","81613C":"Coyote brown",
"FFBCD9":"Cotton candy","FFFDD0":"Cream","DC143C":"Crimson","9E1B32":"Crimson (UA)","F5F5F5":"Cultured","00FFFF":"Cyan","00B7EB":"Cyan (process)","58427C":"Cyber grape","FFD300":"Cyber yellow","F56FA1":"Cyclamen","666699":"Dark blue-gray","654321":"Dark brown","5D3954":"Dark byzantium","26428B":"Dark cornflower blue","008B8B":"Dark cyan","536878":"Dark electric blue","B8860B":"Dark goldenrod","013220":"Dark green","006400":"Dark green (X11)","1A2421":"Dark jungle green","BDB76B":"Dark khaki","483C32":"Dark lava","534B4F":"Dark liver","543D37":"Dark liver (horses)","8B008B":"Dark magenta","4A5D23":"Dark moss green",
"556B2F":"Dark olive green","FF8C00":"Dark orange","9932CC":"Dark orchid","03C03C":"Dark pastel green","301934":"Dark purple","8B0000":"Dark red","E9967A":"Dark salmon","8FBC8F":"Dark sea green","3C1414":"Dark sienna","8CBED6":"Dark sky blue","483D8B":"Dark slate blue","2F4F4F":"Dark slate gray","177245":"Dark spring green","00CED1":"Dark turquoise","9400D3":"Dark violet","00703C":"Dartmouth green","555555":"Davy's grey","DA3287":"Deep cerise","FAD6A5":"Deep champagne","B94E48":"Deep chestnut","004B49":"Deep jungle green","FF1493":"Deep pink","FF9933":"Deep saffron","00BFFF":"Deep sky blue","4A646C":"Deep Space Sparkle","7E5E60":"Deep taupe",
"1560BD":"Denim","2243B6":"Denim blue","C19A6B":"Desert","EDC9AF":"Desert sand","696969":"Dim gray","1E90FF":"Dodger blue","D71868":"Dogwood rose","967117":"Drab","00009C":"Duke blue","EFDFBB":"Dutch white","E1A95F":"Earth yellow","555D50":"Ebony","C2B280":"Ecru","1B1B1B":"Eerie black","614051":"Eggplant","F0EAD6":"Eggshell","1034A6":"Egyptian blue","7DF9FF":"Electric blue","00FF00":"Electric green","6F00FF":"Electric indigo","CCFF00":"Electric lime","BF00FF":"Electric purple","8F00FF":"Electric violet","50C878":"Emerald","6C3082":"Eminence","1B4D3E":"English green",
"B48395":"English lavender","AB4B52":"English red","CC474B":"English vermillion","563C5C":"English violet","00FF40":"Erin","96C8A2":"Eton blue","C19A6B":"Fallow","801818":"Falu red","B53389":"Fandango","DE5285":"Fandango pink","F400A1":"Fashion fuchsia","E5AA70":"Fawn","4D5D53":"Feldgrau","4F7942":"Fern green","6C541E":"Field drab","FF5470":"Fiery rose","B22222":"Firebrick","CE2029":"Fire engine red","E95C4B":"Fire opal","E25822":"Flame","EEDC82":"Flax","0063dc":"Flickr Blue","FB0081":"Flickr Pink","A2006D":"Flirt","FFFAF0":"Floral white","15F4EE":"Fluorescent blue",
"5FA777":"Forest green (Crayola)","014421":"Forest green (traditional)","228B22":"Forest green (web)","A67B5B":"French beige","856D4D":"French bistre","0072BB":"French blue","FD3F92":"French fuchsia","86608E":"French lilac","9EFD38":"French lime","D473D4":"French mauve","FD6C9E":"French pink","C72C48":"French raspberry","F64A8A":"French rose","77B5FE":"French sky blue","8806CE":"French violet","E936A7":"Frostbite","FF00FF":"Fuchsia","C154C1":"Fuchsia (Crayola)","CC397B":"Fuchsia purple","C74375":"Fuchsia rose","E48400":"Fulvous","87421F":"Fuzzy Wuzzy","DCDCDC":"Gainsboro","E49B0F":"Gamboge","007F66":"Generic viridian","F8F8FF":"Ghost white",
"6082B6":"Glaucous","AB92B3":"Glossy grape","00AB66":"GO green","A57C00":"Gold","D4AF37":"Gold (metallic)","FFD700":"Gold (web) (Golden)","E6BE8A":"Gold (Crayola)","85754E":"Gold Fusion","996515":"Golden brown","FCC200":"Golden poppy","FFDF00":"Golden yellow","DAA520":"Goldenrod","676767":"Granite gray","A8E4A0":"Granny Smith apple","808080":"Gray (web)","BEBEBE":"Gray (X11 gray)","00FF00":"Green","1CAC78":"Green (Crayola)","008000":"Green (web)","00A877":"Green (Munsell)","009F6B":"Green (NCS)","00AD43":"Green (Pantone)","00A550":"Green (pigment)","66B032":"Green (RYB)","1164B4":"Green-blue","2887C8":"Green-blue (Crayola)",
"009966":"Green-cyan","A7F432":"Green Lizard","6EAEA1":"Green Sheen","ADFF2F":"Green-yellow","F0E891":"Green-yellow (Crayola)","A99A86":"Grullo","2a3439":"Gunmetal","446CCF":"Han blue","5218FA":"Han purple","E9D66B":"Hansa yellow","3FFF00":"Harlequin","DA9100":"Harvest gold","FF7A00":"Heat Wave","DF73FF":"Heliotrope","AA98A9":"Heliotrope gray","F400A1":"Hollywood cerise","F0FFF0":"Honeydew","006DB0":"Honolulu blue","49796B":"Hooker's green","FF1DCE":"Hot magenta","FF69B4":"Hot pink","355E3B":"Hunter green","71A6D2":"Iceberg","FCF75E":"Icterine","319177":"Illuminating emerald","ED2939":"Imperial red",
"B2EC5D":"Inchworm","4C516D":"Independence","138808":"India green","CD5C5C":"Indian red","E3A857":"Indian yellow","4B0082":"Indigo","00416A":"Indigo dye","002FA7":"International Klein Blue","FF4F00":"International orange (aerospace)","BA160C":"International orange (engineering)","C0362C":"International orange (Golden Gate Bridge)","5A4FCF":"Iris","B3446C":"Irresistible","F4F0EC":"Isabelline","B2FFFF":"Italian sky blue","FFFFF0":"Ivory","00A86B":"Jade","F8DE7E":"Jasmine","A50B5E":"Jazzberry jam","343434":"Jet","F4CA16":"Jonquil","BDDA57":"June bud","29AB87":"Jungle green","4CBB17":"Kelly green","3AB09E":"Keppel","E8F48C":"Key lime",
"C3B091":"Khaki (web)","F0E68C":"Khaki (X11) (Light khaki)","882D17":"Kobe","E79FC4":"Kobi","6B4423":"Kobicha","354230":"Kombu green","512888":"KSU purple","D6CADD":"Languid lavender","26619C":"Lapis lazuli","FFFF66":"Laser lemon","A9BA9D":"Laurel green","CF1020":"Lava","B57EDC":"Lavender (floral)","E6E6FA":"Lavender (web)","CCCCFF":"Lavender blue","FFF0F5":"Lavender blush","C4C3D0":"Lavender gray","7CFC00":"Lawn green","FFF700":"Lemon","FFFACD":"Lemon chiffon","CCA01D":"Lemon curry","FDFF00":"Lemon glacier","F6EABE":"Lemon meringue","FFF44F":"Lemon yellow","FFFF9F":"Lemon yellow (Crayola)","545AA7":"Liberty",
"ADD8E6":"Light blue","F08080":"Light coral","93CCEA":"Light cornflower blue","E0FFFF":"Light cyan","C8AD7F":"Light French beige","FAFAD2":"Light goldenrod yellow","D3D3D3":"Light gray","90EE90":"Light green","FED8B1":"Light orange","C5CBE1":"Light periwinkle","FFB6C1":"Light pink","FFA07A":"Light salmon","20B2AA":"Light sea green","87CEFA":"Light sky blue","778899":"Light slate gray","B0C4DE":"Light steel blue","FFFFE0":"Light yellow","C8A2C8":"Lilac","AE98AA":"Lilac Luster","BFFF00":"Lime (color wheel)","00FF00":"Lime (web) (X11 green)","32CD32":"Lime green","195905":"Lincoln green","FAF0E6":"Linen","C19A6B":"Lion","DE6FA1":"Liseran purple",
"6CA0DC":"Little boy blue","674C47":"Liver","B86D29":"Liver (dogs)","6C2E1F":"Liver (organ)","987456":"Liver chestnut","6699CC":"Livid","FFBD88":"Macaroni and Cheese","CC3336":"Madder Lake","FF00FF":"Magenta","F653A6":"Magenta (Crayola)","CA1F7B":"Magenta (dye)","D0417E":"Magenta (Pantone)","FF0090":"Magenta (process)","9F4576":"Magenta haze","AAF0D1":"Magic mint","F8F4FF":"Magnolia","C04000":"Mahogany","FBEC5D":"Maize","F2C649":"Maize (Crayola)","6050DC":"Majorelle blue","0BDA51":"Malachite","979AAA":"Manatee","F37A48":"Mandarin","FDBE02":"Mango","FF8243":"Mango Tango","74C365":"Mantis",
"880085":"Mardi Gras","EAA221":"Marigold","C32148":"Maroon (Crayola)","800000":"Maroon (web)","B03060":"Maroon (X11)","E0B0FF":"Mauve","915F6D":"Mauve taupe","EF98AA":"Mauvelous","47ABCC":"Maximum blue","30BFBF":"Maximum blue green","ACACE6":"Maximum blue purple","5E8C31":"Maximum green","D9E650":"Maximum green yellow","733380":"Maximum purple","D92121":"Maximum red","A63A79":"Maximum red purple","FAFA37":"Maximum yellow","F2BA49":"Maximum yellow red","4C9141":"May green","73C2FB":"Maya blue","66DDAA":"Medium aquamarine","0000CD":"Medium blue","E2062C":"Medium candy apple red","AF4035":"Medium carmine","F3E5AB":"Medium champagne","BA55D3":"Medium orchid",
"9370DB":"Medium purple","3CB371":"Medium sea green","7B68EE":"Medium slate blue","00FA9A":"Medium spring green","48D1CC":"Medium turquoise","C71585":"Medium violet-red","F8B878":"Mellow apricot","F8DE7E":"Mellow yellow","FEBAAD":"Melon","D3AF37":"Metallic gold","0A7E8C":"Metallic Seaweed","9C7C38":"Metallic Sunburst","E4007C":"Mexican pink","7ED4E6":"Middle blue","8DD9CC":"Middle blue green","8B72BE":"Middle blue purple","8B8680":"Middle grey","4D8C57":"Middle green","ACBF60":"Middle green yellow","D982B5":"Middle purple","E58E73":"Middle red","A55353":"Middle red purple","FFEB00":"Middle yellow","ECB176":"Middle yellow red","702670":"Midnight","191970":"Midnight blue",
"004953":"Midnight green (eagle green)","FFC40C":"Mikado yellow","FFDAE9":"Mimi pink","E3F988":"Mindaro","36747D":"Ming","F5E050":"Minion yellow","3EB489":"Mint","F5FFFA":"Mint cream","98FF98":"Mint green","BBB477":"Misty moss","FFE4E1":"Misty rose","967117":"Mode beige","8DA399":"Morning blue","8A9A5B":"Moss green","30BA8F":"Mountain Meadow","997A8D":"Mountbatten pink","18453B":"MSU green","C54B8C":"Mulberry","C8509B":"Mulberry (Crayola)","FFDB58":"Mustard","317873":"Myrtle green","D65282":"Mystic","AD4379":"Mystic maroon","F6ADC6":"Nadeshiko pink","FADA5E":"Naples yellow","FFDEAD":"Navajo white",
"000080":"Navy blue","1974D2":"Navy blue (Crayola)","4666FF":"Neon blue","39FF14":"Neon green","D7837F":"New York pink","727472":"Nickel","A4DDED":"Non-photo blue","E9FFDB":"Nyanza","4F42B5":"Ocean Blue","48BF91":"Ocean green","CC7722":"Ochre","43302E":"Old burgundy","CFB53B":"Old gold","FDF5E6":"Old lace","796878":"Old lavender","673147":"Old mauve","C08081":"Old rose","848482":"Old silver","808000":"Olive","6B8E23":"Olive Drab (#3)","3C341F":"Olive Drab (#7)","B5B35C":"Olive green","9AB973":"Olivine","353839":"Onyx","A8C3BC":"Opal","B784A7":"Opera mauve",
"FF7F00":"Orange","FF7538":"Orange (Crayola)","FF5800":"Orange (Pantone)","FFA500":"Orange (web)","FF9F00":"Orange peel","FF681F":"Orange-red","FF5349":"Orange-red (Crayola)","FA5B3D":"Orange soda","F5BD1F":"Orange-yellow","F8D568":"Orange-yellow (Crayola)","DA70D6":"Orchid","F2BDCD":"Orchid pink","E29CD2":"Orchid (Crayola)","2D383A":"Outer space (Crayola)","FF6E4A":"Outrageous Orange","800020":"Oxblood","002147":"Oxford blue","841617":"OU Crimson red","1CA9C9":"Pacific blue","006600":"Pakistan green","682860":"Palatinate purple","BCD4E6":"Pale aqua","9BC4E2":"Pale cerulean","FADADD":"Pale pink","FAE6FA":"Pale purple (Pantone)","C9C0BB":"Pale silver",
"ECEBBD":"Pale spring bud","78184A":"Pansy purple","009B7D":"Paolo Veronese green","FFEFD5":"Papaya whip","E63E62":"Paradise pink","50C878":"Paris Green","DEA5A4":"Pastel pink","800080":"Patriarch","536878":"Payne's grey","FFE5B4":"Peach","FFCBA4":"Peach (Crayola)","FFDAB9":"Peach puff","D1E231":"Pear","B768A2":"Pearly purple","CCCCFF":"Periwinkle","C3CDE6":"Periwinkle (Crayola)","E12C2C":"Permanent Geranium Lake","1C39BB":"Persian blue","00A693":"Persian green","32127A":"Persian indigo","D99058":"Persian orange","F77FBE":"Persian pink","701C1C":"Persian plum","CC3333":"Persian red","FE28A2":"Persian rose","EC5800":"Persimmon",
"8BA8B7":"Pewter Blue","DF00FF":"Phlox","000F89":"Phthalo blue","123524":"Phthalo green","2E2787":"Picotee blue","C30B4E":"Pictorial carmine","FDDDE6":"Piggy pink","01796F":"Pine green","2A2F23":"Pine tree","FFC0CB":"Pink","D74894":"Pink (Pantone)","FC74FD":"Pink flamingo","FFDDF4":"Pink lace","D8B2D1":"Pink lavender","F78FA7":"Pink Sherbet","93C572":"Pistachio","E5E4E2":"Platinum","8E4585":"Plum","DDA0DD":"Plum (web)","5946B2":"Plump Purple","5DA493":"Polished Pine","86608E":"Pomp and Power","BE4F62":"Popstar","FF5A36":"Portland Orange","B0E0E6":"Powder blue","F58025":"Princeton orange",
"701C1C":"Prune","003153":"Prussian blue","DF00FF":"Psychedelic purple","CC8899":"Puce","644117":"Pullman Brown (UPS Brown)","FF7518":"Pumpkin","6A0DAD":"Purple","800080":"Purple (web)","9F00C5":"Purple (Munsell)","A020F0":"Purple (X11)","9678B6":"Purple mountain majesty","4E5180":"Purple navy","FE4EDA":"Purple pizzazz","9C51B6":"Purple Plum","9A4EAE":"Purpureus","436B95":"Queen blue","E8CCD7":"Queen pink","A6A6A6":"Quick Silver","8E3A59":"Quinacridone magenta","FF355E":"Radical Red","242124":"Raisin black","FBAB60":"Rajah","E30B5D":"Raspberry","915F6D":"Raspberry glace","B3446C":"Raspberry rose","D68A59":"Raw Sienna",
"826644":"Raw umber","FF33CC":"Razzle dazzle rose","E3256B":"Razzmatazz","8D4E85":"Razzmic Berry","663399":"Rebecca Purple","FF0000":"Red","EE204D":"Red (Crayola)","F2003C":"Red (Munsell)","C40233":"Red (NCS)","ED2939":"Red (Pantone)","ED1C24":"Red (pigment)","FE2712":"Red (RYB)","FF5349":"Red-orange","FF681F":"Red-orange (Crayola)","FF4500":"Red-orange (Color wheel)","E40078":"Red-purple","FD3A4A":"Red Salsa","C71585":"Red-violet","C0448F":"Red-violet (Crayola)","922B3E":"Red-violet (Color wheel)","A45A52":"Redwood","002387":"Resolution blue","777696":"Rhythm","004040":"Rich black","010B13":"Rich black (FOGRA29)","010203":"Rich black (FOGRA39)",
"444C38":"Rifle green","00CCCC":"Robin egg blue","8A7F80":"Rocket metallic","838996":"Roman silver","FF007F":"Rose","F9429E":"Rose bonbon","9E5E6F":"Rose Dust","674846":"Rose ebony","E32636":"Rose madder","FF66CC":"Rose pink","AA98A9":"Rose quartz","C21E56":"Rose red","905D5D":"Rose taupe","AB4E52":"Rose vale","65000B":"Rosewood","D40000":"Rosso corsa","BC8F8F":"Rosy brown","002366":"Royal blue (dark)","4169E1":"Royal blue (light)","7851A9":"Royal purple","FADA5E":"Royal yellow","CE4676":"Ruber","D10056":"Rubine red","E0115F":"Ruby","9B111E":"Ruby red","A81C07":"Rufous",
"80461B":"Russet","679267":"Russian green","32174D":"Russian violet","B7410E":"Rust","DA2C43":"Rusty red","043927":"Sacramento State green","8B4513":"Saddle brown","FF7800":"Safety orange","FF6700":"Safety orange (blaze orange)","EED202":"Safety yellow","F4C430":"Saffron","BCB88A":"Sage","23297A":"St. Patrick's blue","FA8072":"Salmon","FF91A4":"Salmon pink","C2B280":"Sand","967117":"Sand dune","F4A460":"Sandy brown","507D2A":"Sap green","0F52BA":"Sapphire","0067A5":"Sapphire blue","0067A5":"Sapphire (Crayola)","CBA135":"Satin sheen gold","FF2400":"Scarlet","FF91AF":"Schauss pink","FFD800":"School bus yellow",
"66FF66":"Screamin' Green","2E8B57":"Sea green","00FFCD":"Sea green (Crayola)","59260B":"Seal brown","FFF5EE":"Seashell","FFBA00":"Selective yellow","704214":"Sepia","8A795D":"Shadow","778BA5":"Shadow blue","009E60":"Shamrock green","8FD400":"Sheen green","D98695":"Shimmering Blush","5FA778":"Shiny Shamrock","FC0FC0":"Shocking pink","FF6FFF":"Shocking pink (Crayola)","882D17":"Sienna","C0C0C0":"Silver","C9C0BB":"Silver (Crayola)","AAA9AD":"Silver (Metallic)","ACACAC":"Silver chalice","C4AEAD":"Silver pink","BFC1C2":"Silver sand","CB410B":"Sinopia","FF3855":"Sizzling Red","FFDB00":"Sizzling Sunrise","007474":"Skobeloff",
"87CEEB":"Sky blue","76D7EA":"Sky blue (Crayola)","CF71AF":"Sky magenta","6A5ACD":"Slate blue","708090":"Slate gray","299617":"Slimy green","C84186":"Smitten","100C08":"Smoky black","FFFAFA":"Snow","893843":"Solid pink","757575":"Sonic silver","1D2951":"Space cadet","807532":"Spanish bistre","0070B8":"Spanish blue","D10047":"Spanish carmine","989898":"Spanish gray","009150":"Spanish green","E86100":"Spanish orange","F7BFBE":"Spanish pink","E60026":"Spanish red","00FFFF":"Spanish sky blue","4C2882":"Spanish violet","007F5C":"Spanish viridian","A7FC00":"Spring bud","87FF2A":"Spring Frost","00FF7F":"Spring green",
"ECEBBD":"Spring green (Crayola)","007BB8":"Star command blue","4682B4":"Steel blue","CC33CC":"Steel pink","5F8A8B":"Steel Teal","FADA5E":"Stil de grain yellow","E4D96F":"Straw","914E75":"Sugar Plum","FFCC33":"Sunglow","E3AB57":"Sunray","FAD6A5":"Sunset","CF6BA9":"Super pink","A83731":"Sweet Brown","D2B48C":"Tan","D99A6C":"Tan (Crayola)","F28500":"Tangerine","E4717A":"Tango pink","FB4D46":"Tart Orange","483C32":"Taupe","8B8589":"Taupe gray","D0F0C0":"Tea green","F88379":"Tea rose","F4C2C2":"Tea rose","008080":"Teal","367588":"Teal blue","CF3476":"Telemagenta",
"CD5700":"Tenné (tawny)","E2725B":"Terra cotta","D8BFD8":"Thistle","DE6FA1":"Thulian pink","FC89AC":"Tickle Me Pink","0ABAB5":"Tiffany Blue","DBD7D2":"Timberwolf","EEE600":"Titanium yellow","FF6347":"Tomato","00755E":"Tropical rain forest","2D68C4":"True Blue","1C05B3":"Trypan Blue","3E8EDE":"Tufts blue","DEAA88":"Tumbleweed","40E0D0":"Turquoise","00FFEF":"Turquoise blue","A0D6B4":"Turquoise green","8A9A5B":"Turtle green","FAD6A5":"Tuscan","6F4E37":"Tuscan brown","7C4848":"Tuscan red","A67B5B":"Tuscan tan","C09999":"Tuscany","8A496B":"Twilight lavender","66023C":"Tyrian purple","0033AA":"UA blue",
"D9004C":"UA red","3F00FF":"Ultramarine","4166F5":"Ultramarine blue","FF6FFF":"Ultra pink","FC6C85":"Ultra red","635147":"Umber","FFDDCA":"Unbleached silk","5B92E5":"United Nations blue","FFFF66":"Unmellow yellow","014421":"UP Forest green","7B1113":"UP maroon","AE2029":"Upsdell red","AFDBF5":"Uranian blue","004F98":"USAFA blue","664228":"Van Dyke brown","F3E5AB":"Vanilla","F38FA9":"Vanilla ice","C5B358":"Vegas gold","C80815":"Venetian red","43B3AE":"Verdigris","E34234":"Vermilion","D9381E":"Vermilion","A020F0":"Veronica","8F00FF":"Violet","7F00FF":"Violet (color wheel)","963D7F":"Violet (crayola)",
"8601AF":"Violet (RYB)","EE82EE":"Violet (web)","324AB2":"Violet-blue","766EC8":"Violet-blue (Crayola)","F75394":"Violet-red","40826D":"Viridian","009698":"Viridian green","9F1D35":"Vivid burgundy","00CCFF":"Vivid sky blue","FFA089":"Vivid tangerine","9F00FF":"Vivid violet","CEFF00":"Volt","004242":"Warm black","F5DEB3":"Wheat","FFFFFF":"White","A2ADD0":"Wild blue yonder","D470A2":"Wild orchid","FF43A4":"Wild Strawberry","FC6C85":"Wild watermelon","A75502":"Windsor tan","722F37":"Wine","673147":"Wine dregs","FF007C":"Winter Sky","56887D":"Wintergreen Dream","C9A0DC":"Wisteria","C19A6B":"Wood brown",
"EEED09":"Xanthic","738678":"Xanadu","0C020F":"Xiketic","0F4D92":"Yale Blue","FFFF00":"Yellow","FCE883":"Yellow (Crayola)","EFCC00":"Yellow (Munsell)","FFD300":"Yellow (NCS)","FEDF00":"Yellow (Pantone)","FFEF00":"Yellow (process)","FEFE33":"Yellow (RYB)","9ACD32":"Yellow-green","C5E384":"Yellow-green (Crayola)","30B21A":"Yellow-green (Color Wheel)","FFAE42":"Yellow Orange","FF9505":"Yellow Orange (Color Wheel)","FFF700":"Yellow Sunshine","2E5090":"YInMn Blue","0014A8":"Zaffre","39A78E":"Zomp"}


def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def get_color_category(rgb):
    hsv = rgb_to_hsv(rgb[0]/255.0, rgb[1]/255.0, rgb[2]/255.0)
    hue = hsv[0] * 360
    saturation = hsv[1]
    value = hsv[2]

    if saturation < 0.1 and value > 0.9:
        return "White"
    elif value < 0.1:
        return "Black"
    elif saturation < 0.1:
        return "Gray"
    elif 0 <= hue < 15 or 345 <= hue <= 360:
        return "Red"
    elif 15 <= hue < 45:
        return "Orange"
    elif 45 <= hue < 70:
        return "Yellow"
    elif 70 <= hue < 165:
        return "Green"
    elif 165 <= hue < 195:
        return "Cyan"
    elif 195 <= hue < 255:
        return "Blue"
    elif 255 <= hue < 295:
        return "Purple"
    elif 295 <= hue < 345:
        return "Pink"
    else:
        return "Brown"

def create_main_color_groups(colors_dict):
    main_color_groups = {
        "Red": [], "Orange": [], "Yellow": [], "Green": [], "Blue": [],
        "Purple": [], "Pink": [], "Brown": [], "White": [], "Gray": [], "Black": []
    }

    for hex_code, color_name in colors_dict.items():
        try:
            rgb = hex_to_rgb(hex_code)
            category = get_color_category(rgb)
            if category == "Cyan":
                category = "Blue"  # Yhdistämme syaanin siniseen
            main_color_groups[category].append(color_name)
        except (ValueError, KeyError) as e:
            print(f"Ohitetaan virheellinen HEX-arvo: {hex_code} ({color_name})")

    # Poistamme tyhjät kategoriat
    return {k: v for k, v in main_color_groups.items() if v}

# Käyttö:
main_color_groups = create_main_color_groups(colors_dict)

# Tulostetaan ryhmien koot
for category, colors in main_color_groups.items():
    print(f"{category}: {len(colors)} väriä")

# Tulostetaan kokonaismäärä
total_colors = sum(len(colors) for colors in main_color_groups.values())
print(f"\nYhteensä: {total_colors} väriä")

# Esimerkki yhden ryhmän sisällöstä
if "Red" in main_color_groups and main_color_groups["Red"]:
    print("\nEsimerkki 'Red' -ryhmän väreistä:")
    print(main_color_groups["Red"][:10])  # Tulostetaan 10 ensimmäistä punaista väriä
else:
    print("\nPunaisia värejä ei löytynyt.")

import json

def format_color_groups(color_groups):
    formatted = "main_color_groups = {\n"
    for category, colors in color_groups.items():
        formatted += f'    "{category}": [\n'
        # Jaa värit riveille, maksimissaan 4 väriä per rivi
        for i in range(0, len(colors), 4):
            line_colors = colors[i:i+4]
            formatted += '        "' + '", "'.join(line_colors) + '",\n'
        formatted = formatted.rstrip(',\n') + '\n    ],\n'
    formatted = formatted.rstrip(',\n') + '\n}'
    return formatted

# Käytä funktiota main_color_groups sanakirjaan
formatted_groups = format_color_groups(main_color_groups)

# Tulosta muotoiltu sanakirja
print(formatted_groups)

# Jos haluat tallentaa tämän tiedostoon:
with open('main_color_groups.py', 'w') as f:
    f.write(formatted_groups)

print("\nMuotoiltu sanakirja on myös tallennettu 'main_color_groups.py' tiedostoon.")

Red: 138 väriä
Orange: 132 väriä
Yellow: 76 väriä
Green: 118 väriä
Blue: 186 väriä
Purple: 43 väriä
Pink: 130 väriä
White: 21 väriä
Gray: 31 väriä
Black: 5 väriä

Yhteensä: 880 väriä

Esimerkki 'Red' -ryhmän väreistä:
['Amaranth', 'Amaranth red', 'Antique ruby', 'Auburn', 'Tea rose', 'Barn red', 'Big dip o’ruby', 'Bittersweet', 'Bittersweet shimmer', 'Black bean']
main_color_groups = {
    "Red": [
        "Amaranth", "Amaranth red", "Antique ruby", "Auburn",
        "Tea rose", "Barn red", "Big dip o’ruby", "Bittersweet",
        "Bittersweet shimmer", "Black bean", "Black coffee", "Blast-off bronze",
        "Blood red", "Bole", "Brandy", "Brick red",
        "Maroon (Crayola)", "Brink pink", "Oxblood", "Burnished brown",
        "Burnt sienna", "Burnt umber", "Cadmium red", "Candy apple red",
        "Tango pink", "Caput mortuum", "Cardinal", "Carmine",
        "Carnelian", "Catawba", "Cedar Chest", "CG red",
        "Cherry blossom pink", "Chestnut", "Chinese red", "Cinereous",
   

## Color form picture Url script

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from rembg import remove
from sklearn.cluster import KMeans
from collections import Counter
from skimage import color
from tqdm.notebook import tqdm
import pandas as pd
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)

def remove_background(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        input_img = Image.open(BytesIO(response.content)).convert('RGBA')
        output = remove(input_img)
        output_np = np.array(output)
        return output_np
    except Exception as e:
        print(f"Error loading or processing image: {e}")
        return None

def cluster_colors(image, n_clusters=5):
    height, width, channels = image.shape
    pixels = image.reshape(height * width, channels)
    non_white_pixels = pixels[(pixels[:, 0:3] < 240).all(axis=1) & (pixels[:, 3] > 128)]
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    kmeans.fit(non_white_pixels[:, 0:3])
    labels = kmeans.labels_
    label_counts = Counter(labels)
    most_common_label = label_counts.most_common(1)[0][0]
    most_common_color = kmeans.cluster_centers_[most_common_label]
    return most_common_color

def closest_color_name(rgb_color, colors_dict):
    lab_color = color.rgb2lab(np.array([[rgb_color]], dtype=np.uint8) / 255.0)
    lab_colors = {}
    for hex_code, name in colors_dict.items():
        r, g, b = int(hex_code[0:2], 16), int(hex_code[2:4], 16), int(hex_code[4:6], 16)
        lab_colors[name] = color.rgb2lab(np.array([[(r, g, b)]], dtype=np.uint8) / 255.0)
    closest_color = min(lab_colors.keys(), key=lambda c: np.linalg.norm(lab_colors[c] - lab_color))
    return closest_color, next(hex_code for hex_code, name in colors_dict.items() if name == closest_color)

def get_main_color(color_name):
    for main_color, colors in main_color_groups.items():
        if any(sub_color.lower() in color_name.lower() for sub_color in colors):
            return main_color
    return "Other"

# Initialize empty lists to store the color names and HEX codes
color_names = []
color_hex_codes = []
main_colors = []

# Loop through each imageUrl in the DataFrame with a progress bar
for url in tqdm(df['imageUrl'], desc="Processing images"):
    output_np = remove_background(url)

    if output_np is not None:
        most_common_color = cluster_colors(output_np, n_clusters=5)
        most_common_color_name, hex_code = closest_color_name(most_common_color, colors_dict)
        color_names.append(most_common_color_name)
        color_hex_codes.append(hex_code)
        main_color = get_main_color(most_common_color_name)
        main_colors.append(main_color)
    else:
        color_names.append(None)
        color_hex_codes.append(None)
        main_colors.append(None)

# Add the new columns to the DataFrame
df['pic_main_color'] = main_colors
df['pic_color'] = color_names
df['pic_color_hex'] = color_hex_codes

# Display the updated DataFrame
pd.set_option('display.max_colwidth', None)
df[['ean', 'imageUrl', 'pic_main_color', 'pic_color', 'pic_color_hex']]


Processing images:   0%|          | 0/300 [00:00<?, ?it/s]

,ean,imageUrl,pic_main_color,pic_color,pic_color_hex
0,6430063530024,https://public.keskofiles.com/f/k-ruoka/product/6430063530024,Orange,Seal brown,59260B
1,4014086020414,https://public.keskofiles.com/f/k-ruoka/product/4014086020414,Gray,Gainsboro,DCDCDC
2,5060102622215,https://public.keskofiles.com/f/k-ruoka/product/5060102622215,Orange,Van Dyke brown,664228
3,6418654002099,https://public.keskofiles.com/f/k-ruoka/product/6418654002099,Orange,Tenné (tawny),CD5700
4,3052911275326,https://public.keskofiles.com/f/k-ruoka/product/3052911275326,Orange,Sunray,E3AB57
...,...,...,...,...,...
295,761856607934,https://public.keskofiles.com/f/k-ruoka/product/761856607934,Gray,Eerie black,1B1B1B
296,7391603500039,https://public.keskofiles.com/f/k-ruoka/product/7391603500039,Orange,Seal brown,59260B
297,7391835921541,https://public.keskofiles.com/f/k-ruoka/product/7391835921541,Green,Apple green,8DB600
298,3052911169380,https://public.keskofiles.com/f/k-ruoka/product/3052911169380,Red,Dark sienna,3C1414


In [ ]:
# Tallenna dataframe CSV-tiedostona
df.to_csv('dataframe.csv', index=False)

# Lataa tiedosto
from google.colab import files
files.download('dataframe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def count_colors(dictionary):
    total_colors = sum(len(colors) for colors in dictionary.values())
    return total_colors

# Sanakirja 1
main_color_groups_1 = {
    "Red": [
        "Absolute Zero", "Amaranth", "Amaranth (M&P)", "Amaranth pink", "Amaranth purple",
        "Amaranth red", "Antique ruby", "Auburn", "Barn red", "Big dip o'ruby", "Blood red",
        "Brick red", "Burgundy", "Cardinal", "Carmine", "Carmine (M&P)", "Carnelian", "Cerise",
        "Cherry", "Crimson", "Fire engine red", "Folly", "Imperial red", "Maroon", "Oxblood",
        "Red", "Red (Crayola)", "Red (Munsell)", "Red (NCS)", "Red (Pantone)", "Red (pigment)",
        "Red (RYB)", "Redwood", "Rosso corsa", "Ruby", "Rusty red", "Sangria", "Scarlet",
        "Turkey red", "Venetian red", "Vermilion"
    ],
    "Pink": [
        "Baker-Miller pink", "Barbie Pink", "Brink pink", "Champagne pink", "Cherry blossom pink",
        "Congo pink", "Cotton candy", "Flamingo pink", "Hot pink", "Lavender blush", "Magenta",
        "Magenta (Crayola)", "Magenta (dye)", "Magenta (Pantone)", "Magenta (process)",
        "Magenta haze", "Mimi pink", "Nadeshiko pink", "New York pink", "Orchid pink", "Pale pink",
        "Paradise pink", "Pastel pink", "Peach", "Persian pink", "Piggy pink", "Pink", "Pink (Pantone)",
        "Pink flamingo", "Pink lace", "Pink lavender", "Pink Sherbet", "Psychedelic purple", "Rose",
        "Rose bonbon", "Rose pink", "Salmon pink", "Tea rose", "Thulian pink"
    ],
    "Orange": [
        "Alloy orange", "Atomic tangerine", "Bittersweet", "Bittersweet shimmer", "Burnt orange",
        "Cadmium orange", "Candy apple red", "Coral", "Coral pink", "Dark orange", "Deep saffron",
        "International orange (aerospace)", "International orange (engineering)",
        "International orange (Golden Gate Bridge)", "Mandarin", "Mango", "Mango Tango",
        "Orange", "Orange (Crayola)", "Orange (Pantone)", "Orange (web)", "Orange peel",
        "Orange-red", "Orange-red (Crayola)", "Orange soda", "Outrageous Orange", "Peach puff",
        "Persimmon", "Portland Orange", "Pumpkin", "Safety orange", "Safety orange (blaze orange)",
        "Salmon", "Sunset", "Tangelo", "Tangerine", "Tea rose", "Terra cotta", "Tiger's eye"
    ],
    "Yellow": [
        "Arylide yellow", "Banana Mania", "Buff", "Cadmium yellow", "Canary", "Canary yellow",
        "Chartreuse (traditional)", "Citrine", "Cream", "Dandelion", "Ecru", "Flax", "Gold",
        "Gold (metallic)", "Gold (web) (Golden)", "Golden poppy", "Golden yellow", "Goldenrod",
        "Icterine", "Jonquil", "Lemon", "Lemon chiffon", "Lemon curry", "Lemon glacier",
        "Lemon meringue", "Lemon yellow", "Lemon yellow (Crayola)", "Maize", "Maize (Crayola)",
        "Maximum yellow", "Mellow yellow", "Metallic gold", "Mikado yellow", "Mustard", "Naples yellow",
        "Olive", "Olive Drab (#3)", "Olive Drab (#7)", "Olive green", "Olivine", "Peach",
        "Saffron", "School bus yellow", "Selective yellow", "Stil de grain yellow", "Sunglow",
        "Yellow", "Yellow (Crayola)", "Yellow (Munsell)", "Yellow (NCS)", "Yellow (Pantone)",
        "Yellow (process)", "Yellow (RYB)", "Yellow Orange", "Yellow Orange (Color Wheel)",
        "Yellow Sunshine"
    ],
    "Green": [
        "Acid green", "Aero blue", "Amazon", "Ao (English)", "Apple green", "Aquamarine",
        "Army green", "Artichoke", "Asparagus", "Avocado", "Bottle green", "British racing green",
        "Brunswick green", "Bud green", "Cadmium green", "Celadon", "Celadon blue", "Celadon green",
        "Chartreuse (web)", "Dark green", "Dark olive green", "Dark pastel green", "Dark spring green",
        "Dartmouth green", "Emerald", "Erin", "Fern green", "Forest green (Crayola)",
        "Forest green (traditional)", "Forest green (web)", "GO green", "Green", "Green (Crayola)",
        "Green (web)", "Green (Munsell)", "Green (NCS)", "Green (Pantone)", "Green (pigment)",
        "Green (RYB)", "Green-blue", "Green-blue (Crayola)", "Green Lizard", "Green Sheen",
        "Green-yellow", "Green-yellow (Crayola)", "Harlequin", "Hooker's green", "Hunter green",
        "India green", "Jade", "Jungle green", "Kelly green", "Lime (color wheel)", "Lime (web) (X11 green)",
        "Lime green", "Malachite", "Mantis", "Maximum green", "May green", "Mint", "Mint cream",
        "Mint green", "Moss green", "Mountain Meadow", "MSU green", "Myrtle green", "Neon green",
        "Ocean green", "Olive", "Olive Drab (#3)", "Olive Drab (#7)", "Olivine", "Pakistan green",
        "Paris Green", "Pear", "Persian green", "Pistachio", "Reseda green", "Rich black (FOGRA29)",
        "Sacramento State green", "Sap green", "Screamin' Green", "Sea green", "Sea green (Crayola)",
        "Shamrock green", "Sheen green", "Shiny Shamrock", "Spanish green", "Spring bud", "Spring green",
        "Spring green (Crayola)", "Tea green", "Teal", "Tropical rain forest", "Turquoise green",
        "UP Forest green", "Verdigris", "Viridian", "Viridian green", "Yellow-green",
        "Yellow-green (Crayola)", "Yellow-green (Color Wheel)"
    ],
    "Blue": [
        "Aero", "Air superiority blue", "Alice blue", "Azure", "Azure (X11/web color)",
        "Baby blue", "Baby blue eyes", "B'dazzled blue", "Beau blue", "Bleu de France",
        "Blue", "Blue (Crayola)", "Blue (Munsell)", "Blue (NCS)", "Blue (Pantone)",
        "Blue (pigment)", "Blue (RYB)", "Blue bell", "Blue sapphire", "Blue-violet",
        "Blue-violet (Crayola)", "Blue yonder", "Blueberry", "Bluetiful", "Bondi blue",
        "Bright navy blue", "Carolina blue", "Celeste", "Celestial blue", "Cerulean",
        "Cerulean blue", "Cerulean frost", "Cerulean (Crayola)", "CG blue", "Cobalt blue",
        "Columbia Blue", "Cornflower blue", "Dark blue", "Dark electric blue", "Denim",
        "Dodger blue", "Duke blue", "Egyptian blue", "Electric blue", "Eton blue",
        "Federal blue", "Glaucous", "Han blue", "Iceberg", "Indigo", "Indigo dye",
        "Iris", "Light blue", "Light cornflower blue", "Light sky blue", "Majorelle blue",
        "Maya blue", "Midnight blue", "Navy blue", "Navy blue (Crayola)", "Neon blue",
        "Non-photo blue", "Ocean Blue", "Oxford blue", "Pacific blue", "Palatinate blue",
        "Periwinkle", "Periwinkle (Crayola)", "Persian blue", "Phthalo blue", "Picotee blue",
        "Powder blue", "Prussian blue", "Resolution blue", "Royal blue (dark)",
        "Royal blue (light)", "Sapphire", "Sapphire blue", "Sky blue", "Sky blue (Crayola)",
        "Space cadet", "Spanish blue", "Steel blue", "Teal", "Teal blue", "True Blue",
        "Tufts blue", "Turquoise", "Turquoise blue", "Ultramarine", "Ultramarine blue",
        "United Nations blue", "Uranian blue", "USAFA blue", "Yale Blue", "YInMn Blue"
    ],
    "Purple": [
        "African violet", "Amethyst", "Byzantine", "Byzantium", "Chinese violet", "Dark byzantium",
        "Dark purple", "Electric indigo", "Electric purple", "Electric violet", "Eminence",
        "Fuchsia", "Fuchsia (Crayola)", "Fuchsia purple", "Heliotrope", "Indigo", "Iris",
        "Lavender (floral)", "Lavender (web)", "Lavender blue", "Light periwinkle", "Lilac",
        "Magenta", "Magenta haze", "Mauve", "Mauve taupe", "Medium orchid", "Medium purple",
        "Mulberry", "Orchid", "Orchid (Crayola)", "Palatinate purple", "Pansy purple",
        "Pearly purple", "Phlox", "Plum", "Plum (web)", "Pomp and Power", "Psychedelic purple",
        "Purple", "Purple (web)", "Purple (Munsell)", "Purple (X11)", "Purple mountain majesty",
        "Purple pizzazz", "Purple Plum", "Purpureus", "Royal purple", "Twilight lavender",
        "Tyrian purple", "Veronica", "Violet", "Violet (color wheel)", "Violet (crayola)",
        "Violet (RYB)", "Violet (web)", "Violet-blue", "Violet-blue (Crayola)", "Vivid violet",
        "Wisteria"
    ],
    "Brown": [
        "Antique brass", "Beaver", "Bistre", "Bistre brown", "Bole", "Bronze", "Brown",
        "Brown sugar", "Buff", "Burgundy", "Burnt sienna", "Burnt umber", "Cafe au lait",
        "Camel", "Caput mortuum", "Chamoisee", "Chestnut", "Chocolate (traditional)",
        "Chocolate (web)", "Cocoa brown", "Coffee", "Copper", "Copper (Crayola)",
        "Copper penny", "Copper rose", "Coyote brown", "Desert", "Desert sand", "Drab",
        "Earth yellow", "Ecru", "Fallow", "Fawn", "Field drab", "Fuzzy Wuzzy", "Gold Fusion",
        "Golden brown", "Goldenrod", "Harvest gold", "Kobicha", "Lion", "Liver", "Liver (dogs)",
        "Liver chestnut", "Mahogany", "Mummy's Tomb", "Ochre", "Raw Sienna", "Raw umber",
        "Rosy brown", "Russet", "Rust", "Saddle brown", "Sand", "Sand dune", "Sandy brown",
        "Seal brown", "Sepia", "Sienna", "Sinopia", "Smoky black", "Soil", "Tan",
        "Tan (Crayola)", "Taupe", "Taupe gray", "Tenne (tawny)", "Tuscan", "Tuscan brown",
        "Tuscan tan", "Umber", "Van Dyke brown", "Walnut brown", "Wood brown"
    ],
    "White": [
        "Alabaster", "Antique white", "Baby powder", "Beige", "Blond", "Cornsilk", "Cosmic latte",
        "Cream", "Eggshell", "Floral white", "Ghost white", "Ivory", "Linen", "Magnolia",
        "Navajo white", "Old lace", "Pearl", "Seashell", "Snow", "White", "White smoke"
    ],
    "Gray": [
        "Ash gray", "Battleship grey", "Cadet grey", "Charcoal", "Cinereous", "Cool grey",
        "Davy's grey", "Dim gray", "Glaucous", "Granite gray", "Gunmetal", "Heliotrope gray",
        "Independence", "Manatee", "Nickel", "Onyx", "Outer space (Crayola)", "Payne's grey",
        "Pewter Blue", "Platinum", "Quick Silver", "Roman silver", "Silver", "Silver (Crayola)",
        "Silver (Metallic)", "Silver chalice", "Silver sand", "Slate gray", "Sonic silver",
        "Spanish gray", "Taupe gray", "Timberwolf"
    ],
    "Black": [
        "Black", "Black bean", "Black chocolate", "Black coffee", "Black coral", "Black olive",
        "Charleston green", "Ebony", "Eerie black", "Jet", "Licorice", "Onyx", "Rich black",
        "Rich black (FOGRA29)", "Rich black (FOGRA39)", "Smoky black", "Warm black", "Xiketic"
    ]
}

# Sanakirja 2
main_color_groups_2 = {
    "Red": [
        "Amaranth", "Amaranth red", "Auburn", "Barn red", "Big dip o'ruby", "Blood red",
        "Brick red", "Burgundy", "Cardinal", "Carmine", "Carmine (M&P)", "Carnelian",
        "Cerise", "Cherry", "Crimson", "Fire engine red", "Folly", "Imperial red",
        "Maroon", "Oxblood", "Red", "Red (Crayola)", "Red (Munsell)", "Red (NCS)",
        "Red (Pantone)", "Red (pigment)", "Red (RYB)", "Redwood", "Rosso corsa", "Ruby",
        "Rusty red", "Sangria", "Scarlet", "Turkey red", "Venetian red", "Vermilion"
    ],
    "Pink": [
        "Amaranth pink", "Baker-Miller pink", "Barbie Pink", "Blush", "Brink pink",
        "Bubblegum pink", "Carnation pink", "Cerise pink", "Cherry blossom pink",
        "China pink", "Coral pink", "Deep pink", "Fandango pink", "French pink",
        "Fuchsia pink", "Hot pink", "Magenta", "Magenta (Crayola)", "Magenta (dye)",
        "Magenta (Pantone)", "Magenta (process)", "Mauvelous", "Medium violet-red",
        "Mountbatten pink", "Mulberry", "Opera mauve", "Orchid pink", "Pastel pink",
        "Peach", "Pink", "Pink (Pantone)", "Pink lace", "Pink Sherbet", "Puce",
        "Raspberry", "Rose", "Rose bonbon", "Rose pink", "Salmon pink", "Shocking pink",
        "Shocking pink (Crayola)", "Thulian pink", "Tickle Me Pink", "Ultra pink"
    ],
    "Orange": [
        "Alloy orange", "Apricot", "Atomic tangerine", "Bittersweet", "Bright orange",
        "Burnt orange", "Cadmium orange", "Carrot orange", "Coral", "Dark orange",
        "Deep saffron", "International orange (aerospace)", "Mango Tango", "Orange",
        "Orange (Crayola)", "Orange (Pantone)", "Orange (web)", "Orange peel",
        "Orange-red", "Orange-red (Crayola)", "Orange soda", "Peach-orange", "Persimmon",
        "Portland Orange", "Princeton orange", "Pumpkin", "Safety orange", "Tangerine",
        "Tenne (Tawny)"
    ],
    "Yellow": [
        "Arylide yellow", "Aureolin", "Bright yellow (Crayola)", "Cadmium yellow",
        "Canary yellow", "Chartreuse yellow", "Citrine", "Cream", "Dandelion",
        "Flax", "Golden yellow", "Goldenrod", "Jonquil", "Lemon", "Lemon chiffon",
        "Lemon curry", "Lemon glacier", "Lemon meringue", "Lemon yellow",
        "Lemon yellow (Crayola)", "Maize", "Mikado yellow", "Mustard", "Naples yellow",
        "Saffron", "School bus yellow", "Selective yellow", "Stil de grain yellow",
        "Sunglow", "Yellow", "Yellow (Crayola)", "Yellow (Munsell)", "Yellow (NCS)",
        "Yellow (Pantone)", "Yellow (process)", "Yellow (RYB)", "Yellow Sunshine"
    ],
    "Green": [
        "Acid green", "Apple green", "Artichoke green", "Asparagus", "Avocado",
        "Bottle green", "British racing green", "Bud green", "Chartreuse (traditional)",
        "Chartreuse (web)", "Dark green", "Dark olive green", "Dartmouth green",
        "Emerald", "Fern green", "Forest green", "Green", "Green (Crayola)",
        "Green (Munsell)", "Green (NCS)", "Green (Pantone)", "Green (pigment)",
        "Green (RYB)", "Harlequin", "Hooker's green", "Hunter green", "India green",
        "Jade", "Kelly green", "Lawn green", "Lime", "Lime green", "Malachite",
        "Mantis", "Mint", "Mint green", "Moss green", "Olive", "Olive Drab (#3)",
        "Olive Drab (#7)", "Pakistan green", "Paris Green", "Persian green",
        "Phthalo green", "Pistachio", "Russian green", "Sap green", "Screamin' Green",
        "Sea green", "Shamrock green", "Spanish green", "Spring bud", "Spring green",
        "Tea green", "Teal", "Viridian", "Yellow-green"
    ],
    "Blue": [
        "Aero", "Air Force blue", "Air superiority blue", "Alice blue", "Azure",
        "Baby blue", "Blue", "Blue (Crayola)", "Blue (Munsell)", "Blue (NCS)",
        "Blue (Pantone)", "Blue (pigment)", "Blue (RYB)", "Bleu de France", "Cerulean",
        "Cobalt blue", "Cornflower blue", "Cyan", "Dark blue", "Denim", "Dodger blue",
        "Duke blue", "Electric blue", "Indigo", "Iris", "Light blue", "Maya blue",
        "Medium blue", "Midnight blue", "Navy blue", "Neon blue", "Palatinate blue",
        "Periwinkle", "Persian blue", "Phthalo blue", "Powder blue", "Prussian blue",
        "Royal blue", "Sapphire", "Sky blue", "Steel blue", "Teal blue", "True Blue",
        "Turquoise", "Ultramarine", "Yale Blue"
    ],
    "Purple": [
        "Amethyst", "Byzantium", "Eggplant", "Electric indigo", "Electric purple",
        "Electric violet", "Fuchsia", "Grape", "Heliotrope", "Indigo", "Lavender",
        "Lilac", "Magenta haze", "Mauve", "Medium orchid", "Medium purple", "Orchid",
        "Palatinate purple", "Plum", "Purple", "Purple (Munsell)", "Purple (X11)",
        "Rebecca Purple", "Royal purple", "Twilight lavender", "Tyrian purple",
        "Violet", "Violet (color wheel)", "Violet (crayola)", "Violet (RYB)",
        "Violet (web)", "Wisteria"
    ],
    "Brown": [
        "Bole", "Bronze", "Brown", "Buff", "Burgundy", "Burnt sienna", "Burnt umber",
        "Caput mortuum", "Chestnut", "Chocolate", "Cocoa brown", "Coffee", "Copper",
        "Desert sand", "Drab", "Fawn", "Kobicha", "Lion", "Liver", "Mahogany",
        "Maroon", "Raw umber", "Rosy brown", "Russet", "Saddle brown", "Sand",
        "Sandy brown", "Sepia", "Sienna", "Tan", "Taupe", "Umber", "Van Dyke brown"
    ],
    "White": [
        "Antique white", "Beige", "Bone", "Champagne", "Cornsilk", "Cream", "Eggshell",
        "Floral white", "Ghost white", "Ivory", "Linen", "Magnolia", "Navajo white",
        "Off-white", "Pearl", "Seashell", "Snow", "Vanilla", "White"
    ],
    "Gray": [
        "Ash gray", "Battleship grey", "Cadet grey", "Charcoal", "Cinerous",
        "Cool gray", "Davy's grey", "Dim gray", "Glaucous", "Gunmetal",
        "Heliotrope gray", "Light gray", "Manatee", "Old silver", "Platinum",
        "Quick Silver", "Silver", "Slate gray", "Sonic silver", "Taupe gray",
        "Timberwolf"
    ],
    "Black": [
        "Black", "Ebony", "Jet", "Licorice", "Onyx", "Rich black"
    ],
    "Metallic": [
        "Bronze", "Copper", "Gold", "Gold (metallic)", "Metallic gold",
        "Metallic Seaweed", "Metallic Sunburst", "Platinum", "Silver",
        "Silver (Metallic)"
    ]
}


count_1 = count_colors(main_color_groups_1)
count_2 = count_colors(main_color_groups_2)

print(f"Sanakirja 1 sisältää {count_1} väriä.")
print(f"Sanakirja 2 sisältää {count_2} väriä.")

if count_1 > count_2:
    print("Sanakirja 1 on tarkempi (sisältää enemmän värejä).")
elif count_2 > count_1:
    print("Sanakirja 2 on tarkempi (sisältää enemmän värejä).")
else:
    print("Molemmat sanakirjat sisältävät saman määrän värejä.")

Sanakirja 1 sisältää 578 väriä.
Sanakirja 2 sisältää 370 väriä.
Sanakirja 1 on tarkempi (sisältää enemmän värejä).


In [ ]:
# Oletetaan, että colors_dict on jo määritelty

# Lasketaan värien määrä
color_count = len(colors_dict)

# Tulostetaan tulos
print(f"Värien lukumäärä: {color_count}")

Värien lukumäärä: 880


In [ ]:
import webcolors
from colorsys import rgb_to_hsv

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def get_color_category(rgb):
    hsv = rgb_to_hsv(rgb[0]/255.0, rgb[1]/255.0, rgb[2]/255.0)
    hue = hsv[0] * 360
    saturation = hsv[1]
    value = hsv[2]

    if saturation < 0.1 and value > 0.9:
        return "White"
    elif value < 0.1:
        return "Black"
    elif saturation < 0.1:
        return "Gray"
    elif 0 <= hue < 15 or 345 <= hue <= 360:
        return "Red"
    elif 15 <= hue < 45:
        return "Orange"
    elif 45 <= hue < 70:
        return "Yellow"
    elif 70 <= hue < 165:
        return "Green"
    elif 165 <= hue < 195:
        return "Cyan"
    elif 195 <= hue < 255:
        return "Blue"
    elif 255 <= hue < 295:
        return "Purple"
    elif 295 <= hue < 345:
        return "Pink"
    else:
        return "Brown"

def create_main_color_groups(colors_dict):
    main_color_groups = {
        "Red": [], "Orange": [], "Yellow": [], "Green": [], "Blue": [],
        "Purple": [], "Pink": [], "Brown": [], "White": [], "Gray": [], "Black": []
    }

    for hex_code, color_name in colors_dict.items():
        try:
            rgb = hex_to_rgb(hex_code)
            category = get_color_category(rgb)
            if category == "Cyan":
                category = "Blue"  # Yhdistämme syaanin siniseen
            main_color_groups[category].append(color_name)
        except (ValueError, KeyError) as e:
            print(f"Ohitetaan virheellinen HEX-arvo: {hex_code} ({color_name})")

    # Poistamme tyhjät kategoriat
    return {k: v for k, v in main_color_groups.items() if v}

# Käyttö:
main_color_groups = create_main_color_groups(colors_dict)

# Tulostetaan ryhmien koot
for category, colors in main_color_groups.items():
    print(f"{category}: {len(colors)} väriä")

# Tulostetaan kokonaismäärä
total_colors = sum(len(colors) for colors in main_color_groups.values())
print(f"\nYhteensä: {total_colors} väriä")

# Esimerkki yhden ryhmän sisällöstä
if "Red" in main_color_groups and main_color_groups["Red"]:
    print("\nEsimerkki 'Red' -ryhmän väreistä:")
    print(main_color_groups["Red"][:10])  # Tulostetaan 10 ensimmäistä punaista väriä
else:
    print("\nPunaisia värejä ei löytynyt.")

Red: 138 väriä
Orange: 132 väriä
Yellow: 76 väriä
Green: 118 väriä
Blue: 186 väriä
Purple: 43 väriä
Pink: 130 väriä
White: 21 väriä
Gray: 31 väriä
Black: 5 väriä

Yhteensä: 880 väriä

Esimerkki 'Red' -ryhmän väreistä:
['Amaranth', 'Amaranth red', 'Antique ruby', 'Auburn', 'Tea rose', 'Barn red', 'Big dip o’ruby', 'Bittersweet', 'Bittersweet shimmer', 'Black bean']


In [ ]:
import json

def format_color_groups(color_groups):
    formatted = "main_color_groups = {\n"
    for category, colors in color_groups.items():
        formatted += f'    "{category}": [\n'
        # Jaa värit riveille, maksimissaan 4 väriä per rivi
        for i in range(0, len(colors), 4):
            line_colors = colors[i:i+4]
            formatted += '        "' + '", "'.join(line_colors) + '",\n'
        formatted = formatted.rstrip(',\n') + '\n    ],\n'
    formatted = formatted.rstrip(',\n') + '\n}'
    return formatted

# Käytä funktiota main_color_groups sanakirjaan
formatted_groups = format_color_groups(main_color_groups)

# Tulosta muotoiltu sanakirja
print(formatted_groups)

# Jos haluat tallentaa tämän tiedostoon:
with open('main_color_groups.py', 'w') as f:
    f.write(formatted_groups)

print("\nMuotoiltu sanakirja on myös tallennettu 'main_color_groups.py' tiedostoon.")

main_color_groups = {
    "Red": [
        "Amaranth", "Amaranth red", "Antique ruby", "Auburn",
        "Tea rose", "Barn red", "Big dip o’ruby", "Bittersweet",
        "Bittersweet shimmer", "Black bean", "Black coffee", "Blast-off bronze",
        "Blood red", "Bole", "Brandy", "Brick red",
        "Maroon (Crayola)", "Brink pink", "Oxblood", "Burnished brown",
        "Burnt sienna", "Burnt umber", "Cadmium red", "Candy apple red",
        "Tango pink", "Caput mortuum", "Cardinal", "Carmine",
        "Carnelian", "Catawba", "Cedar Chest", "CG red",
        "Cherry blossom pink", "Chestnut", "Chinese red", "Cinereous",
        "Vermilion", "Tea rose", "Copper penny", "Copper red",
        "Copper rose", "Coquelicot", "Cordovan", "Crimson",
        "Crimson (UA)", "Dark liver (horses)", "Dark red", "Dark sienna",
        "Deep chestnut", "Deep taupe", "English red", "English vermillion",
        "Falu red", "Fiery rose", "Firebrick", "Fire engine red",
        "Fire opal", "French ra